In [2]:
from db_connection import run_ddl_query, run_dml_query, run_select_query

#### Before Insert Trigger

In [3]:
run_ddl_query("DROP TRIGGER IF EXISTS prevent_low_salary")

query = """
Create trigger if not exists prevent_low_salary
before insert on emp
for each row
BEGIN
    If NEW.salary < 50 then
        SIGNAL SQLSTATE '45000'
        SET Message_Text = 'Salary must at least be 50';
    END IF;
END;
"""

run_ddl_query(query)

2025-06-10 10:36:12,829 - INFO - DDL executed (e.g., trigger/table created or dropped)


2025-06-10 10:36:12,837 - INFO - DDL executed (e.g., trigger/table created or dropped)


In [4]:
query = """
Insert into emp (eid, ename, salary, deptid) values (101, 'rom', 10, 1)"""

run_dml_query(query)

2025-06-10 10:38:07,856 - ERROR - Error executing DML: (1644, 'Salary must at least be 50')


#### After Insert Trigger 

In [5]:
run_ddl_query("""
CREATE TABLE IF NOT EXISTS emp_audit (
    audit_id INT AUTO_INCREMENT PRIMARY KEY,
    eid INT,
    ename VARCHAR(50),
    salary INT,
    action_time DATETIME DEFAULT NOW()
);
""")


2025-06-10 10:41:50,352 - INFO - DDL executed (e.g., trigger/table created or dropped)


In [6]:
run_ddl_query("DROP TRIGGER IF EXISTS log_high_salary;")

query = """
Create trigger if not exists log_high_salary
after insert on emp
for each row
BEGIN
    If NEW.salary > 10000 then
    Insert into emp_audit (eid, ename, salary)
    values (NEW.eid, NEW.ename, NEW.salary);
    END IF;
END;
"""

run_ddl_query(query)

2025-06-10 10:45:29,604 - INFO - DDL executed (e.g., trigger/table created or dropped)
2025-06-10 10:45:29,613 - INFO - DDL executed (e.g., trigger/table created or dropped)


In [9]:
run_select_query("Select * from emp_audit")

2025-06-10 10:50:20,262 - INFO - SELECT query executed


,audit_id,eid,ename,salary,action_time


In [11]:
run_dml_query("Insert into emp (eid, ename, salary, deptid) values (103, 'roman', 13000, 2)")

2025-06-10 10:50:38,923 - INFO - DML query executed (insert/update/delete)


In [12]:
run_select_query("Select * from emp_audit")

2025-06-10 10:50:41,344 - INFO - SELECT query executed


,audit_id,eid,ename,salary,action_time
0,2,103,roman,13000,2025-06-10 10:50:38


#### After Update

In [22]:
run_ddl_query("""CREATE TABLE IF NOT EXISTS salary_log (
    log_id INT AUTO_INCREMENT PRIMARY KEY,
    eid INT,
    old_salary INT,
    new_salary INT,
    change_time DATETIME DEFAULT NOW()
);
""")

2025-06-10 11:12:31,462 - INFO - DDL executed (e.g., trigger/table created or dropped)


In [17]:
run_ddl_query("DROP TRIGGER IF EXISTS track_salary_change;")

2025-06-10 11:05:27,157 - INFO - DDL executed (e.g., trigger/table created or dropped)


In [23]:
query = """
Create trigger if not exists track_salary_change
after update on emp
for each row
BEGIN
    If NEW.salary != OLD.salary then
    Insert into salary_log (eid, old_salary, new_salary)
    values (OLD.eid, OLD.salary, NEW.salary);
    END IF;
END;
"""

run_ddl_query(query)

2025-06-10 11:12:48,135 - INFO - DDL executed (e.g., trigger/table created or dropped)


In [24]:
run_select_query("select * from salary_log")

2025-06-10 11:12:50,975 - INFO - SELECT query executed


,log_id,eid,old_salary,new_salary,change_time


In [25]:
run_dml_query("Update emp set salary = salary + 500 where eid = 1")

2025-06-10 11:12:53,362 - INFO - DML query executed (insert/update/delete)


In [26]:
run_select_query("select * from salary_log")

2025-06-10 11:12:55,483 - INFO - SELECT query executed


,log_id,eid,old_salary,new_salary,change_time
0,2,1,1500,2000,2025-06-10 11:12:53


#### Before Update

In [27]:
run_ddl_query("Drop trigger if exists enfore_salary_cap")

2025-06-10 11:14:23,542 - INFO - DDL executed (e.g., trigger/table created or dropped)


In [30]:
query = """
Create trigger if not exists enforce_salary_cap
before update on emp
for each row
BEGIN
    IF NEW.salary > OLD.salary * 2 then
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Salary cannot be increased more than 2 times the current salary';
    END IF;
END;
"""

run_ddl_query(query)

2025-06-10 11:16:40,720 - INFO - DDL executed (e.g., trigger/table created or dropped)


In [31]:
run_dml_query("Update emp set salary = 4500 where eid = 1")

2025-06-10 11:17:44,027 - ERROR - Error executing DML: (1644, 'Salary cannot be increased more than 2 times the current salary')


In [32]:
run_dml_query("Update emp set salary = 4000 where eid = 1")

2025-06-10 11:17:54,979 - INFO - DML query executed (insert/update/delete)


In [33]:
run_select_query("select * from emp where eid = 1")

2025-06-10 11:18:15,148 - INFO - SELECT query executed


,eid,ename,salary,deptid
0,1,a,4000,1


#### Before Delete

In [38]:
run_ddl_query("Drop trigger if exists prevent_fin_emp_deletion")

2025-06-10 11:23:19,727 - INFO - DDL executed (e.g., trigger/table created or dropped)


In [40]:
query = """
Create trigger if not exists prevent_fin_emp_deletion
before delete on emp
for each row
BEGIN
    If OLD.deptid = 3 then
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'You cannot delete an emp from finance dept';
    END IF;
END;
"""
run_ddl_query(query)

2025-06-10 11:23:56,394 - INFO - DDL executed (e.g., trigger/table created or dropped)


In [41]:
run_dml_query("Delete from emp where eid = 3")

2025-06-10 11:25:00,395 - ERROR - Error executing DML: (1644, 'You cannot delete an emp from finance dept')


In [42]:
run_dml_query("Delete from emp where eid = 10")

2025-06-10 11:25:23,332 - INFO - DML query executed (insert/update/delete)


#### After Delete

You cannot delete a row in dept if it's referenced in emp — unless specified otherwise using ON DELETE CASCADE, SET NULL, etc.

In [43]:
run_ddl_query("DROP TABLE IF EXISTS emp;")

2025-06-10 11:35:32,630 - INFO - DML query executed (insert/update/delete)


In [44]:
run_ddl_query("""
CREATE TABLE emp (
    eid INT PRIMARY KEY,
    ename VARCHAR(50) NOT NULL,
    salary INT CHECK (salary > 0),
    deptid INT NOT NULL,
    FOREIGN KEY (deptid) REFERENCES dept(deptid) ON DELETE CASCADE
);
""")

2025-06-10 11:36:03,408 - INFO - DDL executed (e.g., trigger/table created or dropped)


In [45]:
run_dml_query("""
INSERT INTO emp (eid, ename, salary, deptid) VALUES
(1,'a',1000,1), (2,'b',1023,2), (3,'c',23,3), (4,'d',123,3),
(5,'e',102,1), (6,'f',2023,3), (7,'g',323,1), (8,'h',523,2),
(9,'i',136,1), (10,'j',627,1), (11,'k',978,3), (12,'l',222,1),
(13,'m',746,1), (14,'n',3223,1), (15,'o',1999,3), (16,'p',821,3),
(17,'q',324,1), (18,'r',546,3), (19,'s',717,2), (20,'t',817,2),
(21,'u',888,3), (22,'v',9029,1), (23,'w',20001,3), (24,'x',28222,1),
(25,'y',19212,3), (26,'z',12111,1), (27,'a1',3211,1), (28,'a2',9238,1),
(29,'a3',653,3), (30,'a4',44,1), (31,'a5',3,2), (32,'a6',16,1),
(33,'a7',14,2), (34,'a8',845,1), (35,'a9',135,3), (36,'b1',1212,1),
(37,'b2',1242,2), (38,'b3',4212,2), (39,'b4',4444,3), (40,'b5',555,1),
(41,'b6',1553,3), (42,'b7',6432,1), (43,'b8',4532,2), (44,'b9',3355,1),
(45,'c1',2324,2);
""")

2025-06-10 11:36:30,440 - INFO - DML query executed (insert/update/delete)


In [46]:
run_ddl_query("""
CREATE TABLE IF NOT EXISTS deleted_depts_log (
    deptid INT,
    deptname VARCHAR(50),
    deleted_at DATETIME DEFAULT NOW()
);
""")

2025-06-10 11:36:50,872 - INFO - DDL executed (e.g., trigger/table created or dropped)


In [48]:
run_ddl_query("DROP TRIGGER IF EXISTS log_deleted_dept;")

2025-06-10 11:37:08,231 - INFO - DDL executed (e.g., trigger/table created or dropped)


In [50]:
query = """
CREATE TRIGGER log_deleted_dept
AFTER DELETE ON dept
FOR EACH ROW
BEGIN
    INSERT INTO deleted_depts_log (deptid, deptname)
    VALUES (OLD.deptid, OLD.deptname);
END;
"""
run_ddl_query(query)

2025-06-10 11:38:05,800 - INFO - DDL executed (e.g., trigger/table created or dropped)


In [51]:
run_dml_query("INSERT INTO dept (deptid, deptname) VALUES (10, 'TEST');")

2025-06-10 11:38:20,294 - INFO - DML query executed (insert/update/delete)


In [52]:
run_dml_query("""
INSERT INTO emp (eid, ename, salary, deptid) VALUES
(100, 'TestEmp1', 2000, 10),
(101, 'TestEmp2', 2500, 10);
""")

2025-06-10 11:38:35,970 - INFO - DML query executed (insert/update/delete)


In [56]:
run_dml_query("DELETE FROM dept WHERE deptid = 10;")

2025-06-10 11:40:07,186 - INFO - DML query executed (insert/update/delete)


In [59]:
run_select_query("select * from deleted_depts_log;")

2025-06-10 11:40:36,342 - INFO - SELECT query executed


,deptid,deptname,deleted_at
0,10,TEST,2025-06-10 11:38:55


In [60]:
run_select_query("SELECT * FROM emp WHERE deptid = 10;")

2025-06-10 11:40:41,005 - INFO - SELECT query executed


,eid,ename,salary,deptid
